# Kullback Leibler Divergence

## Imports

In [1]:
import numpy as np
import pickle
import pandas as pd
import spacy

from collections import Counter 
from pathlib import Path
from scipy.special import kl_div

## Constant Paths

In [2]:
# PATHS: INFILES

# Path to directory containing preprocessed COCA files
COCA_PREPROC_DIR = Path("./coca-preproc-spacy/")

# Path to directory containing preprocessed Elsevier files
ELSEVIER_PREPROC_DIR = Path("./elsevier-preproc-spacy/")

# Path to file containing all subject areas
SUBJAREAS = Path("./subjareas.txt")

## Load COCA
Since the COCA data will be used throughout the entire process, it will be loaded now for ease.

In [3]:
with open(f'{COCA_PREPROC_DIR}/2015.pickle', 'rb') as f:
    coca = pickle.load(f)

print(len(coca))

7802


## Calculating KLD
In the following section all functions necessary to conduct a KLD analysis for both token unigrams and POS trigrams are defined. We also define a function to conduct bootstrapping. Since stopwords are excluded during the token unigram analysis, we also load all stopwords.

In [7]:
# Load stopwods to filter out when counting token frequency distributions
nlp = spacy.load("en_core_web_sm")
stopwords = nlp.Defaults.stop_words

In [27]:
def token_freq_dist(text_sample, text_key, min_occ = 10):
    """
    Calculates the frequency distribution of tokens within a text sample.
    
    Args:
    text_sample: A list of preprocessed texts.
    text_key: A string describing the key for where body text is stored for each article object. 

    Returns: A dictionary with each token as key and frequency as value. 
    """
    all_tokens = []

    for text in text_sample:
        text_tokens = [token.text for sentence in text[text_key] for token in sentence]
        filter_tokens = [token.lower() for token in text_tokens if token not in stopwords]

        all_tokens.extend(filter_tokens)

    tokens_counts = dict(Counter(all_tokens))
    filtered_counts = [
        (token, count) for token, count in tokens_counts.items() if count >= min_occ
        ]

    return filtered_counts

def pos_trigram_freq_dist(text_sample, text_key, min_occ = 10):
    """Calculates the frequency distribution of pos trigrams within a text sample.
    
    Args:
    text_sample: A list of preprocessed texts.
    text_key: A string describing the key for where body text is stored for each 
    article object. 

    Returns: A dictionary with each trigram as key and frequency as value. 
    """

    all_trigrams = []
 
    for text in text_sample:
        pos_tags = [[token.tag_ for token in sentence] for sentence in text[text_key]]
        pos_trigrams = ['-'.join(sentence[token:token+3]) 
                        for sentence in pos_tags 
                        for token in range(len(sentence)-2)]

        all_trigrams.extend(pos_trigrams)

    trigram_counts = dict(Counter(all_trigrams))
    filtered_counts = [
        (trigram, count) for trigram, count in trigram_counts.items() if count >= min_occ
        ]

    return filtered_counts

def kld(p, q):
    "Calculate the Kullback-Leibler Divergence between two frequency distributions p and q."

    p_keys = [item[0] for item in p]
    q_keys = [item[0] for item in q]

    tokens = list(set(p_keys).union(set(q_keys)))

    # Ensure the same items in both lists and the same order
    p_ordered = [
        (k, [t[1] for t in p if t[0] == k][0]) if k in p_keys else (k, 0) for k in tokens
        ]
    q_ordered = [
        (k, [t[1] for t in q if t[0] == k][0]) if k in q_keys else (k, 0) for k in tokens
        ]
    
    p_ordered_values = np.array([item[1] for item in p_ordered])
    q_ordered_values = np.array([item[1] for item in q_ordered])

    # Normalize the arrays to obtain probability distributions
    p_dist = p_ordered_values / p_ordered_values.sum()
    q_dist = q_ordered_values / q_ordered_values.sum()

    pointwise_kld = {}

    for i in range(len(p_ordered_values)):
        if q_dist[i] == 0:
            pointwise_kld[q_ordered[i][0]] = kl_div(p_dist[i], np.exp(-9))
        else:
            pointwise_kld[q_ordered[i][0]]= kl_div(p_dist[i], q_dist[i])

    kld = np.sum(list(pointwise_kld.values()))
    
    return kld, pointwise_kld

def bootstrap_kld_ci(
        baseline_text_dist, 
        comparison_texts, 
        num_resamples=1000, 
        alpha=0.05, 
        versus = 0,
        type = "token"
        ):
    """
    Calculates KLD for two corpora, with bootstrapping one one the distributions.
    It calculates KLD for either token distribution of POS trigram distributions.

    Args:
    baseline_text_dist: A frequency distribution of either tokens of POS trigrams
    (depending on 'type'), for which the resampled corpora will be compared to. 
    comparison_texts: A corpora containing preprocessed text documents that will be 
    bootstrapped.
    num_resamples: The number of iterations for bootstrappign. Default = 1000.
    alpha: The alpha value for which to calculate the confidence intervals. Default = 0.05.
    verus: Determines whether to treat baseline_text_dist as the dependent or independent 
    corpus in the calculation. Defaul = 0, indicating comparison texts vs. baseline texts.
    type: Determine whether to calculate KLD for tokens or POS trigrams. Default = "token".
    """
    kl_divergences = []
    n = len(comparison_texts)

    for i in range(num_resamples):
        resampled_texts = np.random.choice(comparison_texts, size=n, replace=True)

        if type == "token":
            resampled_dist = token_freq_dist(resampled_texts, "body_text_docs")
        else: 
            resampled_dist = pos_trigram_freq_dist(resampled_texts, "body_text_docs")
        
        if versus == 0:
            kl_divergence_i = kld(resampled_dist, baseline_text_dist)
        else:
            kl_divergence_i = kld(baseline_text_dist, resampled_dist)

        kl_divergences.append(kl_divergence_i[0])

    average_kld = np.mean(kl_divergences)    
    ci_lower = np.percentile(kl_divergences, alpha/2 * 100)
    ci_upper = np.percentile(kl_divergences, (1 - alpha/2) * 100)

    return average_kld, ci_lower, ci_upper

## KLD for Token Unigrams

In [5]:
# Define dataframes to store results
subj_coca_token_df = pd.DataFrame(columns=['subj', 'sum', 'lower CI', 'upper CI', 'pointwise KLD'])
coca_subj_token_df = pd.DataFrame(columns=['subj', 'sum', 'lower CI', 'upper CI', 'pointwise KLD'])

In [8]:
# Calculate token frequency distribution in COCA
coca_token_dist = token_freq_dist(coca, 'text_docs')

In [24]:
# Number of unique tokens in filtered COCA.
len(coca_token_dist)

42896

We run through each academic discipline in the Elsevier OA CC-BY corpus for which the pointwise KLD is calculated for the entire corpus and bootstrapping yields 95% confidence intervals and an estimated KLD average for the corpus. The KLD is calculated in both directions.

In [30]:
with open(f'{SUBJAREAS}', 'r') as subj_list_file:
    for subject in subj_list_file:
        subject = subject.strip()
        print(subject)

        with open(f'{ELSEVIER_PREPROC_DIR}/{subject}.pickle', 'rb') as articles_file:
            articles = pickle.load(articles_file)
        
        subj_token_dist = token_freq_dist(articles, 'body_text_docs')

        # Subject vs. Coca
        bootstrap_subj_coca = bootstrap_kld_ci(coca_token_dist, articles, num_resamples= 1, alpha=0.05, versus = 0)
        pointwise_subj_coca = kld(subj_token_dist, coca_token_dist)[1]
        
        subj_coca_row = {
            'subj': subject, 'sum': bootstrap_subj_coca[0], 'lower CI': bootstrap_subj_coca[1], 
            'upper CI': bootstrap_subj_coca[2], 'pointwise KLD': pointwise_subj_coca
            }
        
        subj_coca_token_df = subj_coca_token_df.append(subj_coca_row, ignore_index=True)

        # Coca vs. Subject
        bootstrap_coca_subj = bootstrap_kld_ci(coca_token_dist, articles, num_resamples= 1, alpha=0.05, versus = 1)
        pointwise_coca_subj = kld(subj_token_dist, coca_token_dist)[1]

        coca_subj_row = {
            'subj': subject, 'sum': bootstrap_coca_subj[0], 'lower CI': bootstrap_coca_subj[1], 
            'upper CI': bootstrap_coca_subj[2], 'pointwise KLD': pointwise_coca_subj
            }
        
        coca_subj_token_df = coca_subj_token_df.append(coca_subj_row, ignore_index=True)


## KLD for POS Trigrams
Once again, we first count the POS-trigram distributions. Then we calculate the KLD and associated confidence intervalss per subject though bootstrapping. Pointwise KLD is also calculated using the entire corpus of each academic discipline. 

In [ ]:
# Define dataframes to store results
subj_coca_pos_df = pd.DataFrame(columns=['subj', 'sum', 'lower CI', 'upper CI', 'pointwise KLD'])
coca_subj_pos_df = pd.DataFrame(columns=['subj', 'sum', 'lower CI', 'upper CI', 'pointwise KLD'])

In [8]:
coca_pos_dist = pos_trigram_freq_dist(coca, 'text_docs')

In [33]:
with open(f'{SUBJAREAS}', 'r') as subj_list_file:
    for subject in subj_list_file:
        subject = subject.strip()
        print(subject)

        with open(f'{ELSEVIER_PREPROC_DIR}/{subject}.pickle', 'rb') as articles_file:
            articles = pickle.load(articles_file)
        
        subj_pos_dist = pos_trigram_freq_dist(articles, 'body_text_docs')

        # Subject vs. Coca
        subj_coca_kld = kld(subj_pos_dist, coca_pos_dist)
        print(f'subject_coca_kld is:{subj_coca_kld[0]}')
        subj_coca_ci = bootstrap_kld_ci(coca_token_dist, articles, num_resamples= 1000, alpha=0.05, versus = 0)
        
        subj_coca_row = {
            'subj': subject, 'sum': bootstrap_subj_coca[0], 'lower CI': bootstrap_subj_coca[1], 
            'upper CI': bootstrap_subj_coca[2], 'pointwise KLD': pointwise_subj_coca
            }
        subj_coca_pos_df = subj_coca_pos_df.append(subj_coca_row, ignore_index=True)

        # Coca vs. Subject
        coca_subj_kld = kld(coca_pos_dist, subj_pos_dist)
        print(f'coca_subject_kld is {coca_subj_kld[0]}')
        coca_subj_ci = bootstrap_kld_ci(coca_token_dist, articles, num_resamples= 1000, alpha=0.05, versus = 1)

        coca_subj_row = {
            'subj': subject, 'sum': bootstrap_coca_subj[0], 'lower CI': bootstrap_coca_subj[1], 
            'upper CI': bootstrap_coca_subj[2], 'pointwise KLD': pointwise_coca_subj
            }
        coca_subj_pos_df = coca_subj_pos_df.append(coca_subj_row, ignore_index=True)